In [2]:
#import necessary packages
import numpy as np
import pandas as pd


First we can download in our own directory the files regarding:
1) medication and 
2) updated brain files.

# Files to download ########


scp user@beluga.computecanada.ca:/lustre03/project/6055423/UK_Biobank/Files_CSV/0_UKB_ATC_Medications.csv /Users/computer1

scp user@beluga.computecanada.ca:/lustre03/project/6055423/UK_Biobank/Files_CSV/UKB_NoBrain_19K_V3.csv  /Users/computer1

scp user@beluga.computecanada.ca:/lustre03/project/6055423/UK_Biobank/Files_CSV/Fan_Cluster_279_0-10000.csv.gz /Users/computer1/UKB_Files/

# Merge the same files

This part took 30 mins to read

In [10]:
home_path ='/Users/ellenwang/Documents/RA_Work_Doc_EvpLab/tmp_data/'

# read in the medication data
med = pd.read_csv('0_UKB_ATC_Medications.csv') 
#files for the nodes
edges = pd.read_csv('Fan_Cluster_279_Edges_Details.csv')  
edge_dic = dict(zip(edges['index'], edges.names))


UKB = pd.read_csv('UKB_NoBrain_19K_V3.csv') 
UKB_brain = pd.read_csv('Fan_Cluster_279_0-10000.csv.gz')
UKB_brain = UKB_brain.rename(columns = edge_dic)
UKB = pd.read_csv(home_path + 'UKB_NoBrain_19K_V3.csv') 
UKB_brain = pd.read_csv(home_path + 'Fan_Cluster_279_0-10000.csv.gz')
UKB_brain = UKB_brain.rename(columns = edge_dic)



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1744,1746,1748,1749,1750,1751,1778,1779,1780,1781,1794,1795,1796,1798,1830,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1965,1966,1967,1968,1969,1972,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2177,2178,2179,2180,2916,2917,2932,2933,2934,2935,2936,2937,2938,2939,2967,2968,2969,2970,2971,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,3011,3012,3013,3049,3050,3051,3052,3053,3054,

In [4]:
UKB_brain2 = pd.read_csv(home_path + 'Fan_Cluster_279_10000-19797.csv.gz').rename(columns = edge_dic)

UKB_brain3 = UKB_brain.append(UKB_brain2)





# Select opioids

In [5]:
#select opioids
med['OPs'] = med.filter(like='_T2').filter(like='N02A').sum(axis=1)
med['OPs'] = np.where(med['OPs'].values > 0, 1, 0)
med['SRIs'] = med.filter(like='_T2').filter(like='N06A').sum(axis=1)
med['SRIs'] = np.where(med['SRIs'].values > 0, 1, 0)
med['Anlg'] = med.filter(like='_T2').filter(like='N02').sum(axis=1)
med['Anlg'] = np.where(med['Anlg'].values > 0, 1, 0)
med['NumberMed'] = med.filter(like='_T2').sum(axis = 1)


In [6]:
 #Note if you want other analgesics, you can select N02B or N02C. 
#This takes the sum of values. This will be useful for us to get antidepressants.
UKB2 = UKB.merge(med[['OPs','SRIs','Anlg', 'eid','NumberMed']], on = 'eid')


# Select the limbic brain

In [7]:


### Select the limbic brain, total of 9000 edges.
select = np.all(edges[['E1', 'E2']].isin(['Fr','Ins','BS','SbC','Lim']), axis = 1)
Limbic = UKB_brain[edges['names'][select].values]

# Fisher r-to-z score by normalizing
Limbic = pd.DataFrame(data = np.arctanh(Limbic.values), columns = Limbic.columns) 
Limbic['eid'] = UKB_brain['eid'] # I would recommend always merging on eid

Limbic2 = Limbic.merge(UKB2[['SRIs', 'OPs','Anlg','NumberMed',\
                             'NumberPainTypesLastMonth_T2', 'ChronicPain_T2',\
                             'NumberChronicPainTypes_T2','Age_T2','Sex_T0_x', 'eid']], on = 'eid')


In [8]:
Limbic2

,Fr_SFG-L (8m)_x_Fr_SFG-R (8m),Fr_SFG-L (8m)_x_Fr_SFG-L (8dl),Fr_SFG-R (8m)_x_Fr_SFG-L (8dl),Fr_SFG-L (8m)_x_Fr_SFG-R (8dl),Fr_SFG-R (8m)_x_Fr_SFG-R (8dl),Fr_SFG-L (8dl)_x_Fr_SFG-R (8dl),Fr_SFG-L (8m)_x_Fr_SFG-L (9l),Fr_SFG-R (8m)_x_Fr_SFG-L (9l),Fr_SFG-L (8dl)_x_Fr_SFG-L (9l),Fr_SFG-R (8dl)_x_Fr_SFG-L (9l),...,eid,SRIs,OPs,Anlg,NumberMed,NumberPainTypesLastMonth_T2,ChronicPain_T2,NumberChronicPainTypes_T2,Age_T2,Sex_T0_x
0,0.379658,0.469283,-0.417853,-0.023965,0.270102,0.003460,0.621426,-0.297791,1.559072,-0.013741,...,2835807,0,0,0,0,1,0,0.0,50.750000,1.0
1,0.402551,0.674099,-0.191621,0.010050,-0.284668,0.698241,1.207374,0.081188,0.925268,0.192565,...,2058315,0,0,0,0,2,1,1.0,51.000000,0.0
2,0.716329,0.226222,-0.346322,-0.243957,-0.538036,0.824996,0.350321,-0.190387,0.845669,0.273082,...,4612421,0,0,0,0,0,0,0.0,63.166667,0.0
3,0.249172,0.225391,-0.994017,-0.358123,-0.618955,0.310102,0.121504,-0.475099,0.518563,0.382758,...,1439140,0,0,0,0,0,0,0.0,60.083333,0.0
4,0.490635,0.060383,-0.637947,-0.116940,-0.414563,0.780058,0.083755,-0.754760,0.808533,0.658419,...,4792907,0,0,0,0,2,1,2.0,49.250000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,0.104448,0.676700,-0.051866,0.129042,0.185085,0.425937,0.471784,-0.057052,0.860218,-0.072869,...,3131688,0,0,1,6,3,1,3.0,49.416667,0.0
9990,0.510584,0.152522,-0.474068,0.124997,0.005960,0.772568,0.272210,-0.499673,1.390587,0.545620,...,5717305,1,0,1,9,5,1,4.0,52.833333,0.0
9991,0.847555,0.715028,0.194174,-0.267514,-0.457580,0.095338,0.853936,0.337283,0.831723,-0.423018,...,2346004,0,0,0,0,0,0,0.0,69.083333,1.0
9992,1.242968,0.548933,0.311202,0.073955,0.010760,0.719872,0.136186,-0.057042,0.713409,0.275710,...,1656019,0,0,0,1,0,0,0.0,75.750000,1.0


In [9]:
UKB_brain

,Fr_SFG-L (8m)_x_Fr_SFG-R (8m),Fr_SFG-L (8m)_x_Fr_SFG-L (8dl),Fr_SFG-R (8m)_x_Fr_SFG-L (8dl),Fr_SFG-L (8m)_x_Fr_SFG-R (8dl),Fr_SFG-R (8m)_x_Fr_SFG-R (8dl),Fr_SFG-L (8dl)_x_Fr_SFG-R (8dl),Fr_SFG-L (8m)_x_Fr_SFG-L (9l),Fr_SFG-R (8m)_x_Fr_SFG-L (9l),Fr_SFG-L (8dl)_x_Fr_SFG-L (9l),Fr_SFG-R (8dl)_x_Fr_SFG-L (9l),...,Cb_Cb-R (IX)_x_BS_RN-L,Cb_Cb-L (X)_x_BS_RN-L,Cb_Cb-V (X)_x_BS_RN-L,Cb_Cb-R (X)_x_BS_RN-L,SbC_HypoTha_x_BS_RN-L,BS_BS_x_BS_RN-L,BS_PAG_x_BS_RN-L,BS_SN-L_x_BS_RN-L,BS_SN-R_x_BS_RN-L,eid
0,0.36241,0.43762,-0.39512,-0.02396,0.26372,0.00346,0.55212,-0.28929,0.91527,-0.01374,...,0.33900,0.01981,0.28667,0.05336,0.11120,0.43730,-0.15889,0.08604,-0.11873,2835807
1,0.38213,0.58767,-0.18931,0.01005,-0.27722,0.60325,0.83589,0.08101,0.72838,0.19022,...,0.08709,-0.14898,0.04061,0.19165,0.15015,0.17474,0.13549,0.18156,0.18410,2058315
2,0.61463,0.22244,-0.33311,-0.23923,-0.49150,0.67778,0.33666,-0.18812,0.68880,0.26649,...,0.50269,-0.08010,0.12871,-0.14647,0.06117,0.64027,0.19382,-0.22727,-0.10990,4612421
3,0.24414,0.22165,-0.75907,-0.34356,-0.55040,0.30053,0.12091,-0.44231,0.47659,0.36510,...,0.51093,0.33979,0.33306,0.37284,0.01095,0.28196,0.65793,0.12025,0.46692,1439140
4,0.45472,0.06031,-0.56350,-0.11641,-0.39234,0.65274,0.08356,-0.63798,0.66878,0.57731,...,-0.35641,0.16549,-0.07492,-0.00565,-0.06478,-0.00747,-0.04146,-0.07688,-0.01077,4792907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.10407,0.58937,-0.05182,0.12833,0.18300,0.40192,0.43964,-0.05699,0.69637,-0.07274,...,0.12671,0.13006,0.35129,0.04802,0.24262,0.33752,0.32251,0.21735,0.40587,3131688
9996,0.47040,0.15135,-0.44148,0.12435,0.00596,0.64842,0.26568,-0.46186,0.88330,0.49723,...,0.05984,0.04932,0.13781,0.07522,0.28479,-0.08386,0.02155,-0.18325,-0.06848,5717305
9997,0.68979,0.61382,0.19177,-0.26131,-0.42811,0.09505,0.69312,0.32505,0.68140,-0.39947,...,0.10997,-0.07529,-0.01559,0.17046,-0.17731,0.33240,0.26370,0.18388,-0.04114,2346004
9998,0.84630,0.49972,0.30153,0.07382,0.01076,0.61683,0.13535,-0.05698,0.61281,0.26893,...,-0.04726,-0.09815,0.16798,-0.02807,0.12028,0.54940,0.19878,0.20329,0.43069,1656019


In [20]:
#####Computing average connectivity within the limbic system for each parcellation


df = UKB_brain #file with 10,000 subjects with edges renamed for the parcellations they contain
nodes_df = pd.read_csv('Fan_Cluster_279_Details.csv') #name of each parcel
edges = pd.read_csv('Fan_Cluster_279_Edges_Details.csv')#name of each edge

select = np.all(edges[['E1', 'E2']].isin(['Fr','Ins','BS','SbC','Lim']), axis = 1)
Limbic = df[edges['names'][select].values]
Limbic = pd.DataFrame(data = np.arctanh(Limbic.values), columns = Limbic.columns) # Fisher r-to-z score by normalizing
Limbic['eid'] = df['eid'] # I would recommend always merging on eid

nodes = nodes_df['regions']#get parcel names

df_new = pd.DataFrame()#make a new dataframe that i’ll fill with average connectivity values


In [13]:
Limbic 

,Fr_SFG-L (8m)_x_Fr_SFG-R (8m),Fr_SFG-L (8m)_x_Fr_SFG-L (8dl),Fr_SFG-R (8m)_x_Fr_SFG-L (8dl),Fr_SFG-L (8m)_x_Fr_SFG-R (8dl),Fr_SFG-R (8m)_x_Fr_SFG-R (8dl),Fr_SFG-L (8dl)_x_Fr_SFG-R (8dl),Fr_SFG-L (8m)_x_Fr_SFG-L (9l),Fr_SFG-R (8m)_x_Fr_SFG-L (9l),Fr_SFG-L (8dl)_x_Fr_SFG-L (9l),Fr_SFG-R (8dl)_x_Fr_SFG-L (9l),...,SbC_Tha-L (cTtha)_x_BS_RN-L,SbC_Tha-R (cTtha)_x_BS_RN-L,SbC_Tha-L (lPFtha)_x_BS_RN-L,SbC_Tha-R (lPFtha)_x_BS_RN-L,SbC_HypoTha_x_BS_RN-L,BS_BS_x_BS_RN-L,BS_PAG_x_BS_RN-L,BS_SN-L_x_BS_RN-L,BS_SN-R_x_BS_RN-L,eid
0,0.379658,0.469283,-0.417853,-0.023965,0.270102,0.003460,0.621426,-0.297791,1.559072,-0.013741,...,0.144007,-0.042085,0.281886,0.091605,0.111662,0.468888,-0.160248,0.086253,-0.119293,2835807
1,0.402551,0.674099,-0.191621,0.010050,-0.284668,0.698241,1.207374,0.081188,0.925268,0.192565,...,0.230708,0.052809,0.019122,0.040502,0.151294,0.176552,0.136328,0.183595,0.186223,2058315
2,0.716329,0.226222,-0.346322,-0.243957,-0.538036,0.824996,0.350321,-0.190387,0.845669,0.273082,...,0.750236,0.149780,0.145202,0.253419,0.061246,0.758631,0.196303,-0.231309,-0.110346,4612421
3,0.249172,0.225391,-0.994017,-0.358123,-0.618955,0.310102,0.121504,-0.475099,0.518563,0.382758,...,0.119861,-0.019883,0.340048,0.343715,0.010950,0.289810,0.789155,0.120835,0.506124,1439140
4,0.490635,0.060383,-0.637947,-0.116940,-0.414563,0.780058,0.083755,-0.754760,0.808533,0.658419,...,-0.032541,0.046363,0.066709,-0.133273,-0.064871,-0.007470,-0.041484,-0.077032,-0.010770,4792907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.104448,0.676700,-0.051866,0.129042,0.185085,0.425937,0.471784,-0.057052,0.860218,-0.072869,...,0.238205,0.282384,0.266033,0.288236,0.247556,0.351291,0.334446,0.220873,0.430657,3131688
9996,0.510584,0.152522,-0.474068,0.124997,0.005960,0.772568,0.272210,-0.499673,1.390587,0.545620,...,-0.188667,-0.155245,0.007320,-0.162136,0.292887,-0.084057,0.021553,-0.185344,-0.068587,5717305
9997,0.847555,0.715028,0.194174,-0.267514,-0.457580,0.095338,0.853936,0.337283,0.831723,-0.423018,...,0.157940,0.109587,0.067332,0.430226,-0.179204,0.345524,0.270081,0.185996,-0.041163,2346004
9998,1.242968,0.548933,0.311202,0.073955,0.010760,0.719872,0.136186,-0.057042,0.713409,0.275710,...,0.330912,0.158986,0.290364,0.396465,0.120865,0.617522,0.201462,0.206162,0.460744,1656019


# CENTRALITY ATTEMPT

# Computing average connectivity within the limbic system for each parcellation



# Getting Cohen’s d between average connectivity of Opioid users and HC

# PLS ATTEMPT

In [14]:
!pip install pyfiglet
!pip install termcolor
!pip install termcolor
!pip install colorama

import sys

from colorama import init
init(strip=not sys.stdout.isatty()) # strip colors if stdout is redirected
from termcolor import cprint
from pyfiglet import figlet_format

cprint(figlet_format('Learning Git is fun!', font='starwars'),'yellow', 'on_red', attrs=['bold'])



You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
 __       _______     ___      .______      .__   __.  __  .__   __.   _______ 
|  |     |   ____|   /   \     |   _  \     |  \ |  | |  | |  \ |  |  /  _____|
|  |     |  |__     /  ^  \    |  |_)  |    |   \|  | |  | |   \|  | |  |  __  
|  |     |   __|   /  /_\  \   |      /     |  . `  | |  | |  . `  | |  | |_ | 
|  `----.|  |____ /  _____  \  |  |\  \----.|  |\   | |  | |  |\   | |  |__| | 
|_______||_______/__/     \__\ |